<a href="https://colab.research.google.com/github/simransirsat/MLPracticals/blob/main/Gain_Ratio.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**5A Gain Ratio**

In [1]:
pip install anytree

     |████████████████████████████████| 51kB 2.0MB/s 


In [2]:
import pandas as pd

In [3]:
from math import log2
from anytree import Node, RenderTree

In [4]:
df = pd.read_csv('/content/sample_data/data.csv')

In [5]:
def Info(df):
    possible_labels = {}
    for i in range(0, df.shape[0]):
        if df.iloc[i, -1] not in possible_labels:
            possible_labels[df.iloc[i, -1]] = 1
        else:
            possible_labels[df.iloc[i, -1]] += 1
    info = 0
    total_records = df.shape[0]
    for label, count in possible_labels.items():
        info -= (count / total_records) * log2((count / total_records))

    assert (info >= 0), "Info(D) is negative"
    return [round(info,4), possible_labels]

def entropy(df, attr, attr_label, possible_values):
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i

    label_cnts = {}
    for i in range(0, df.shape[0]):
        if(df.iloc[i, columns[attr]] == attr_label):
            if df.iloc[i, -1] not in label_cnts:
                label_cnts[df.iloc[i, -1]] = 1
            else:
                label_cnts[df.iloc[i, -1]] += 1
    info = 0
    for label, count in label_cnts.items():
        info -= (count / possible_values[attr_label]) * log2((count / possible_values[attr_label]))
    return round(info,4)

def Info_attr(df, attr):
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i

    possible_values = {}        # (attr_labels, cnt) : (<30. 4)
    for i in range(0, df.shape[0]):
        if df.iloc[i, columns[attr]] not in possible_values:
            possible_values[df.iloc[i, columns[attr]]] = 1
        else:
            possible_values[df.iloc[i, columns[attr]]] += 1
    
    info_attr = 0
    for attr_label, cnt in possible_values.items():
        info_attr += (cnt / df.shape[0]) * entropy(df, attr, attr_label, possible_values)
        print("\t", attr_label, "->", entropy(df, attr, attr_label, possible_values))
    return round(info_attr,4)

def split_info(df, attr):
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i
    possible_values = {}
    for i in range(0, df.shape[0]):
        if df.iloc[i, columns[attr]] not in possible_values:
            possible_values[df.iloc[i, columns[attr]]] = 1
        else:
            possible_values[df.iloc[i, columns[attr]]] += 1

    splitinfo = 0
    for attr_label, cnt in possible_values.items():
        splitinfo -= (cnt / df.shape[0]) * log2((cnt/ df.shape[0]))
    return round(splitinfo,4)

def find_deciding_attr(df):
    max_gain_ratio = float('-inf')
    max_gain_ratio_attr = None
    
    print('Info_df:\t', Info(df)[0])
    for i in range(0, df.shape[1] - 1):
        print('\n', df.columns[i])
        info_attr = Info_attr(df, df.columns[i])
        print('Info(', df.columns[i],') :', round(info_attr,4))
        gain = Info(df)[0] - info_attr
        print('GAIN:\t', round(gain,4))
        splitinfo = split_info(df, df.columns[i])
        print('Split info:\t', round(splitinfo,4))
        if splitinfo == 0:
            print('Skipping column: ',df.columns[i],' since splitinfo is 0')
            continue
        gainratio = gain / splitinfo
        print('Gain Ratio\t', round(gainratio,4))

        if(gainratio > max_gain_ratio):
            max_gain_ratio = gainratio
            max_gain_ratio_attr = df.columns[i]
        elif(gainratio == max_gain_ratio):
            if df.columns[i] < max_gain_ratio_attr:
                max_gain_ratio_attr = df.columns[i]

    if max_gain_ratio_attr is None:
        print(df)
    #assert max_gain_ratio_attr is not None, "Attribute finding error, max_gain_ratio_attr not set"
    print('\n\nDeciding Attribute: ', max_gain_ratio_attr,' with gain ratio= ', round(max_gain_ratio,4))

    return max_gain_ratio_attr

root = None

def make_decision_tree(df, parent = None, weight = None):
    global root    
    possible_target = {}
    print(df)
    for i in range(0, df.shape[0]):
        if df.iloc[i, -1] not in possible_target:
            possible_target[df.iloc[i, -1]] = True
    
    if (len(possible_target) == 1):
        node = Node(list(possible_target)[0], parent = parent, weight = str(weight))
        return

    if(df.shape[1] == 1 and len(possible_target) > 1):
        node = Node(list(possible_target)[0], parent = parent, weight = str(weight))
        return
    
    deciding_attr = find_deciding_attr(df)
    if deciding_attr is None:
        node = Node(df.iloc[:, -1].value_counts().idxmax(), parent = parent, weight = str(weight))
        return
        
    node = Node(deciding_attr, parent = parent, weight = str(weight))
    #print('df:', df)
    if (root is None):
        root = node
    
    columns = {}
    for i in range(0, len(list(df.columns))):
        columns[list(df.columns)[i]] = i
    
    possible_values = {}
    for i in range(0, df.shape[0]):
        if df.iloc[i, columns[deciding_attr]] not in possible_values:
            possible_values[df.iloc[i, columns[deciding_attr]]] = True

    #print('possible_values: ', possible_values)
    for attr_wt_label, _ in possible_values.items():
        print('Now checking for ',deciding_attr,':\t', attr_wt_label)
        cols = list(df.columns)
        cols.remove(deciding_attr)
        df_temp = pd.DataFrame(columns = cols)
        
        for i in range(0, df.shape[0]):
            if (df.iloc[i, columns[deciding_attr]] == attr_wt_label):
                row = df.iloc[i, :].tolist()
                row.pop(columns[deciding_attr])
                #df_temp.loc[len(df)] = row
                df_temp = df_temp.append(pd.DataFrame([row], columns = cols), ignore_index = True)
        
        make_decision_tree(df_temp, node, attr_wt_label)

    return

In [6]:
root = None
make_decision_tree(df)

from anytree import RenderTree
for pre, _, node in RenderTree(root):
    print("%s%s (%s)" % (pre, node.name, node.weight))

   Restaurant       Meal       Day       Cost Vegan AllergicReaction?
0       Sam's  Breakfast  Saturday  Expensive   Yes               Yes
1       Kim's  Breakfast    Sunday      Cheap   Yes                No
2       Sam's      Lunch  Saturday      Cheap   Yes               Yes
3       Kim's  Breakfast    Sunday  Expensive    No                No
4       Sam's  Breakfast  Saturday      Cheap    No                No
5       Kim's      Lunch    Friday  Expensive    No               Yes
6       Kim's      Lunch  Saturday      Cheap   Yes                No
7       Kim's  Breakfast    Sunday  Expensive    No               Yes
8       Sam's  Breakfast  Saturday      Cheap    No                No
9       Kim's  Breakfast    Friday  Expensive    No               Yes
10      Sam's      Lunch  Saturday      Cheap    No                No
11      Kim's      Lunch    Friday      Cheap    No                No
Info_df:	 0.9799

 Restaurant
	 Sam's -> 0.971
	 Kim's -> 0.9852
Info( Restaurant ) : 0.97